<a href="https://colab.research.google.com/github/fedric95/YoloTests/blob/main/Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jun 19 06:53:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#import locale
#locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!pip install neptune

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
config = """
# Example usage: python train.py --data ssdd.yaml
# parent
# ├── yolov5
# └── datasets
#     └── ssdd  ← downloads here (7 MB)


# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../datasets/ssdd # dataset root dir
train: train/images # train images (relative to 'path')
val: valid/images   # val images (relative to 'path')
test: # test images (optional)

# Classes
names:
  0: ship


# Download script/URL (optional)
download: https://github.com/fedric95/YoloTests/raw/main/ssdd.zip



augmentations:
- Albumentations:
    blur_limit: 700
    brightness_contrast:
      brightness_limit: 0.1
      contrast_limit: 0.1
    crop:
      height: 1024
      width: 1024
    gauss_noise:
      var_limit: (10.0, 50.0)
    horizontal_flip: true
    hue_saturation:
      hue_shift_limit: 20
      sat_shift_limit: 50
    motion_blur:
      blur_limit: 7

"""

model = """

# Parameters
nc: 1  # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024]  # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  s: [0.33, 0.50, 1024]  # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
  m: [0.67, 0.75, 768]   # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
  l: [1.00, 1.00, 512]   # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  x: [1.00, 1.25, 512]   # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]  # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]  # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]  # cat backbone P4
  - [-1, 3, C2f, [512]]  # 12

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]  # cat backbone P3
  - [-1, 3, C2f, [256]]  # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]]  # cat head P4
  - [-1, 3, C2f, [512]]  # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]]  # cat head P5
  - [-1, 3, C2f, [1024]]  # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]]  # Detect(P3, P4, P5)
"""

f = open("/content/dataset.yaml", "w")
f.write(config)
f.close()

f = open("/content/model.yaml", "w")
f.write(model)
f.close()

In [63]:
import importlib
import ultralytics.yolo.data.augment
importlib.reload(ultralytics.yolo.data.augment)



<module 'ultralytics.yolo.data.augment' from '/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/data/augment.py'>

In [64]:
import neptune
import os
from ultralytics import YOLO

#os.environ["NEPTUNE_API_TOKEN"] = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwMzEyZDhiMC0xZWIxLTRjYzctODcxMi04ZmUyYzc1MmQ1ZDQifQ=="
#os.environ["NEPTUNE_PROJECT"]   = "federico.ricciuti/YOLOv8"

model = YOLO('/content/model.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
model.train(
    data='/content/dataset.yaml',
    epochs=20,
    imgsz=640,
    hsv_h = 0,
    hsv_s = 0,
    hsv_v = 0
)

WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295

In [65]:
# Evaluate the model's performance on the validation set
results = model.val()

Ultralytics YOLOv8.0.119 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
model summary (fused): 168 layers, 3005843 parameters, 0 gradients
val: Scanning /content/datasets/ssdd/valid/labels.cache... 212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 212/212 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:08<00:00,  1.69it/s]
                   all        212        497      0.936       0.94      0.979      0.705
Speed: 2.5ms preprocess, 5.2ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs/detect/val4


In [69]:
import matplotlib.pyplot as plt

# Perform object detection on an image using the model
results = model.predict(
    source = '/content/datasets/ssdd/valid/images/000012_jpg.rf.55a194724e0e7923d38b81b3af595f59.jpg',
    imgsz=640,
    conf = 0.50
)

for result in results:
  plt.imshow(result.plot())
  plt.savefig('output.png')
  plt.show()

  orig_img = result.orig_img
  plt.imshow(orig_img)
  plt.savefig('input.png')
  plt.show()



image 1/1 /content/datasets/ssdd/valid/images/000012_jpg.rf.55a194724e0e7923d38b81b3af595f59.jpg: 640x640 3 ships, 15.2ms
Speed: 16.7ms preprocess, 15.2ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)


In [67]:
!pwd

/content
